<center><h1>7SSG2059 Geocomputation 2018/19</h1></center>

<h1><center>Practical 7: Making Maps</h1></center>


# Working with GeoPandas and PySAL

We've worked -- and will continue to work -- a lot with pandas, but by default pandas doesn't help us much when want to start working with explicitly geographical data. Ways of working _computationally_ with things like distance and location are increasingly important not only to geographers, but also to data scientists, and this is where we start to move away from purely aspatial statistical analysis into the foundations of a more _geographic_ data science.

There are a huge number of modules (a.k.a. packages, a.k.a. libraries) in Python designed to help you work with geodata, but we are going to focus on the two most important higher-level libraries (since they also provide 'wrappers' around some of the lower-level libraries):

1. [GeoPandas](http://geopandas.org/) -- which offers a pandas-like interface to working with geodata. Think of this as your tool for basic data manipulation and transformation, much like pandas. You will almost certainly want to [bookmark the documentation](http://geopandas.org/data_structures.html#geodataframe).
2. [PySAL](http://pysal.readthedocs.io/en/latest/) -- the Python Spatial Analysis Library  provides the spatial analytic functions that we'll need for everything from classification, clustering and point-pattern analysis to autocorrelation-based tools.
3. There is some overlap between the two libraries: both can do plotting but, for reasons we'll see later, we'll normally do this in PySAL and both can do classification (remember, we did _quantiles_ with pandas!) but (again), for reasons we'll see later, we'll often do this in PySAL from here on out.

PySAL is complicated enough that the best way to understand how it fits together is to use an image:

![PySAL Logo](http://darribas.org/gds_scipy16/content/figs/pysal.png)

We're going to concentrate primarily on the ESDA (Exploratory _Spatial_ Data Analysis) and Weights components of PySAL in this module, but you should know about the other bits!

## Required Preamble

It makes life a lot easier if you make all of the library import commands and configuration information (here having to do with `matplotlib`) the first exectuable code in a notebook or script. That way it's easy to see what you need to have installed before you get started!

In [ ]:
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.cm as cm

In [ ]:
import warnings 
warnings.simplefilter('ignore')

The two new packages we're using this week are PySAL and GeoPandas:

In [ ]:
import pysal as ps
import geopandas as gpd

## Creating a GeoPandas DataFrame

There are two primary ways in which we can create a GeoPandas DataFrame:
1. Use an existing Pandas dataframe that contains a `geometry` Series
2. Load a file containing some kind geometry (and possibly data)

As the LSOA data we have been using already has a `geometry` series (column), we'll use approach 1. first, then look at 2. later.

So, first let's read our LSOA Data as a Pandas DataFrame as usual (though you could _also_ read this directly into geopandas): 

In [ ]:
pdf = pd.read_csv( # pdf == pandas dataframe
    'https://github.com/kingsgeocomp/geocomputation/blob/master/data/LSOA%20Data.csv.gz?raw=true',
    compression='gzip', low_memory=False) # The 'low memory' option means pandas doesn't guess data types

print(pdf.columns)

Notice in our Pandas DataFrame that one of our columns (Series) is named _geometry_. This contains information on the shape of each LSOA; let's take a look at it:

In [ ]:
print(pdf.geometry.head())

So each element of this Series has text indicating the type of shape the geometry applies to (e.g. _POLYGON_) followed by a bunch of numbers. These numbers are truncated in the view above, so let's look in a little more detail at a single entry in the _geometry_ column:    

In [ ]:
print(pdf.geometry.iloc[0])

Now we see that what we have is pairs of numbers, each separated by a comma. Each pair of values is a set of corodinates for each the [vertex](https://en.wikipedia.org/wiki/Vertex_(geometry)) of the [polygon](https://en.wikipedia.org/wiki/Polygon). GeoPandas can use this information about the type of shape and the co-ordinates to plot the data as a map (and perform other spatial analysis functions).

So now we can create a GeoPandas DataFrame directly from our Pandas DataFrame:

In [ ]:
gdf = gpd.GeoDataFrame(pdf) # gdf == geopandas dataframe

Next, we need to tell GeoPandas which Series contains the spatial information: 

In [ ]:
gdf = gdf.set_geometry('geometry')

Did you get an error? Can you work out why? [This thread](https://gis.stackexchange.com/questions/267801/csv-to-geodataframe-how-to-have-valid-geometry-objects) might help you solve this problem below...

#### Task 1

The problem here is that _geometry_ column is a `string` type, but it needs to be a `shapely.geometry` type (details [here](http://toblerity.org/shapely/manual.html) if you want to find out more). We need to use pandas `apply` functionality to apply some function to every Polygon string:

In [ ]:
print('Geopandas geometry type: ' + str(type(gdf.geometry)))

from shapely.wkt import loads 
gdf['geometry'] = gdf['geometry'].apply(???)

print('Geopandas geometry type: ' + str(type(gdf.geometry)))

#### What have we done?

We started with a regular pandas data frame and now have somehow ended up with a geopandas data frame, but how?
- `gpd.GeoDataFrame(pdf)` took a pandas dataframe (pdf) and passed it to the geopandas 'constructor' which creates new geopandas dataframes.
- However, that step doesn't mean that we yet have a _geometry_ that can be mapped/analysed, so we still need to tell geopandas where to find the geodata.
- And it turns out that in the 'raw' form that we loaded it the `POLYGON(...)` data is treated as a String. This actually makes sense since pandas doesn't know anything about geography; so it gets to this column and 'thinks' "Ok, well I guess this must be some kind of text..."
- And geopandas responds with "Woah, I don't map text, I only deal with _geometries_."
- So we use the `shapely` library to convert the String to a Geometry object and we use `apply` (a pandas function) to apply it to every value in the `geometry` column and overwrite the original string.
- The last thing we need to know is tell geopandas that it's now a 'proper' geometry...

In [ ]:
gdf = gdf.set_geometry('geometry')

Hopefully there was no error this time!

In [ ]:
gdf.plot()

#### A Quick Revisiting of Inheritance

Lt's check we understand how GeoPandas _inherits_ functionality from Pandas. First, let's check what data type `gdf` is using the [`isinstance`](https://www.w3schools.com/python/ref_func_isinstance.asp) function:

In [ ]:
if isinstance(gdf, gpd.GeoDataFrame): # Is gdf a GeoDataFrame object?
    print("\tI'm a geopandas data frame!")

if isinstance(gdf, pd.DataFrame): # Is gdf *also* a DataFrame object?
    print("\tI'm also a pandas data frame!")

So this is interesting: `gdf` is _both_ a Pandas DataFrame _and_ a GeoPandas DataFrame. This means we can use both Pandas and GeoPandas methods on `gdf`. Here are some Pandas methods:

In [ ]:
print("What is the lsoa11nm column type: ")
print('\tNAME type: ' + str(type(gdf.LSOA11NM)))

In [ ]:
print(gdf.Owned.describe())

In [ ]:
gdf.sample(3, random_state=42)

This can get a bit confusing, but the next two plots demonstrate something quite interesting (and powerful):

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
gdf.MedianPrice.plot.hist(ax=ax)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 10))
gdf.plot(column='MedianPrice', ax=ax)

Hopefully driven home the fact that we can use _both_ the new geopandas functionality and the existing pandas functionality with our geopandas dataframe!

## Making Choropleth Maps!

So now we know how to make something much more interesting by telling GeoPandas to colour the polygons according to some value, like one of the variables in the data: 

In [ ]:
gdf.plot(???)

Cool! But what do the colours mean? We can add a legend like this:

In [ ]:
gdf.plot(column='HHOLDS', legend=???)

Adding things like a plot title or legend labels isn't quite so easy. We'll take a look at how to do this below, but if you take this simple approach for plots in your reports you will need to make sure you are very clear in your figure captions what the figure shows (e.g. including what the units for the numbers are).   

We can also change the shade of colours used using the `cmap` (colormap) option: 

In [ ]:
gdf.plot(column='HHOLDS', legend=True, cmap='OrRd')

For a full list of colormaps possible, see the [matplotlib website](http://matplotlib.org/users/colormaps.html). There are [numerous resources online](https://tilemill-project.github.io/tilemill/docs/guides/tips-for-color/) to help you think about the most appropriate map colouring for the data you want to visualise. The [colorbrewer](http://colorbrewer2.org) tool is particularly useful and [this overview](https://www.e-education.psu.edu/geog486/node/1867) is also helpful. The viridis colormap is [used by default](http://bids.github.io/colormap/) and one of the best all-round colour schemes for [numerous](https://stats.stackexchange.com/a/223324) [reasons](https://www.youtube.com/watch?v=xAoljeRJ3lU). 

The four main matplotlib colormaps are:

[![colors](https://matplotlib.org/_images/sphx_glr_colormaps_001.png)](https://matplotlib.org/users/colormaps.html)

But there are many others:

[![colors](https://matplotlib.org/_images/sphx_glr_colormaps_006.png)](https://matplotlib.org/users/colormaps.html)

Alternative to the continuous colouring approach used above, we can also classify the way colour maps are scaled using the `scheme` option. For example:

In [ ]:
gdf.plot(column='HHOLDS', legend=True, scheme='quantiles', k=3)

The `scheme` option uses functionality from the [PySAL](https://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html) package and can be one of the following three types:
1. `equal_interval`
2. `quantiles`
3. `fisher_jenks`

In each case, we can provide an additional parameter `k` to the plot method to specify how many classes should be created. For example, above we used `k=3`.

You'll note that the legend is overlapping the map, which is not ideal, but we'll deal with that later. 

#### Task 2

1. Create an equal interval map with 6 classes for the _SocialRented_ variable using the _magma_ colormap. Do not include a legend. Set the figure size to 8 by 6, and the dpi to 200. Try again with the dpi set to 100.


In [ ]:
???

2. Create a map of the _Owned_ variable, classified into quintiles. Change the colormap to inferno and include a legend.

In [ ]:
???

## Revisiting Classification

Classification matters. So before we continue with learning about the various mapping options we need to think more about this. 

I realise that this topic can seem a little... _dry_, but perhaps this video will give you some sense of how the choices you make about representing your data can significantly alter your understanding of the data.

[![Do maps lie?](http://img.youtube.com/vi/G0_MBrJnRq0/0.jpg)](http://www.youtube.com/watch?v=G0_MBrJnRq0)

PySAL has a classification 'engine' that we can use to bin data based on attribute values. Although we have just seen that it is possible to do classification _without_ PySAL, and it is also possible to do some mapping _without_ GeoPands, the combination of the two is simplest: PySAL has additional classification methods that are _specific_ to geographic analysis problems, and GeoPandas is just plain easier to work with. 

Let's see how (spatial) classification in PySAL is equivalent to the (a-spatial) classification we've done previously in Pandas by calculating and mapping quintiles. 

First, calculate the quintiles using the Pandas `quantiles` method:

In [ ]:
Quintiles = gdf.Owned.quantile([0,0.2,0.4,0.6,0.8,1])
print(Quintiles)

Remember that quntiles split the data into five parts containing roughly equal proportions of the data. 

So from the output above you should be able to see, for example, that in the lowest 20% of LSOAs fewer than `199` households are owned by their occupier.

We could also [calculate these values using PySAL](https://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html#pysal.esda.mapclassify.Quantiles) (as `GeoPandas` does), but PySAL expects `numpy` arrays instead of a pandas Series. Fortunately, it's easy to convert one to the other since `Pandas`/`GeoPandas` are built on top of `numpy` arrays...)

In [ ]:
import mapclassify as mp

owned_q5 = mp.Quantiles(pdf.Owned.values, k=5) # Classify into 5 quantiles using the 'values' because that is a numpy array
print(owned_q5) # Print summary metrics

There's more information in PySAL's output and we can see how many LSOAs have fallen into each class - it's roughly the same number in each class, which is good because that's the point of quantiles. 

To visualise this distribution numerically, let's plot a histogram of these quantiles using a Seaborn `displot` with the `Pandas` DataFrame:

In [ ]:
sns.distplot(pdf.Owned, bins=Quintiles, norm_hist=True, kde=False) # Notice norm_hist!

In this histogram we forced the y-axis to be the 'frequency density' so that we can see the relative frequency of each of the classes (known as 'bins' in a histogram and shown as the height of the bars). For example, we can see that the bin with highest frequency is the third quintile that contains all the LSOAs with values of 'Owned' between `291` and `366` (where did I get those values from)? 

You should also be able to see where the first bin stops at around `199` (because remember we saw above that the first qunatile is all values up to `199`). 

What if we had plotted the absolute frequency (i.e. the count) of LSOAs in each bin on the y-axis instead of the frequency density: 

In [ ]:
sns.distplot(pdf.Owned, bins=Quintiles, kde=False) # Notice no norm_hist!

Think about why all the bars are roughly the same height... it's because by definition each quintile should contain roughly the same number of LSAOs (i.e. 20% in each bin). 

It's difficult to see where one bar stops and another begins, so let's add some lines to help:

In [ ]:
# What is going on here? Add some notes
sns.distplot(pdf.Owned, bins=Quintiles, kde=False)
for b in Quintiles:
    plt.vlines(b, 0, 1000, color='red', linestyle='--') 

#### Task 3

1. Okay so, we've now throughly examined the _numerical_ distribution of data for the _Owned_ variable, but **where** are the LSOAs in each of those five bins? What is the **spatial** distribution? Where are the LSOAs in the lowest quintile (i.e. LSOAs with _Owned_ values of 5 to 199) vs those in the highest quintile (i.e. LSOAs with _Owned_ values of 452 to 710)?

In [ ]:
???

Remember that `GeoPandas` is using the `PySAL` functionality to do its classification. 

See how the values in the legend match those in the quintiles we calculated above! And so now we can talk about how the numeric distribution we saw in the histograms above has fallen out spatially. 

2. In the box below _describe_ the spatial distribution of 'Owned' households across London using the quantile maps; you may find it helpful to go back and modify the output so that you can see it in more detail by making it larger...

> Your answer here!

But how do we know that equal intervals is the _best_ way to create our classes for visualisation?

One of the best-known geographical classification is Fisher-Jenks (also sometimes known as 'Natural Breaks'), which groups data into bins based on the sum of squared deviations between classes: in other words, the algorithm iteratively looks for ways to group the data into a specified number of bins such that moving a data point from one group to another would increase the total within-class deviation observed in the data.

Let's try the same as above but using (five) Fisher Jenks breaks. Pandas cannot do this so we can only [use PySAL here](https://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html#pysal.esda.mapclassify.Fisher_Jenks) (_**this may take some time depending on your laptop!**_):

In [ ]:
owned_fj = mp.Fisher_Jenks(gdf.Owned.values, k=5) # Classify into Natural Breaks
print(owned_fj) # Print summary metrics

Hopefully you can see how the five classes created using this method are different from the quantiles methods we used above. Note how they are unequal in size, for example.

Let's create a histogram to visualise the numeric distribution of the Fisher Jenks classification. First we need to get the values to create the bins in the histogram. Looking at the example in [the documentation for the PySAL `mapclassify.Fisher_Jenks` function](https://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html#pysal.esda.mapclassify.Fisher_Jenks) we can see that we can access the bins direct from the object

In [ ]:
print(owned_fj.bins)

But note that these are the upper limits of the bin, so we need to add the very lowest value. [We can do this](https://stackoverflow.com/a/36998277/10219907) using the `numpy` `insert` function:

In [ ]:
owned_fjb = np.insert(owned_fj.bins, 0, gdf.Owned.min())  # insert the minimum value into the zeroth position of the array 
print(owned_fjb)   # print to check what it looks like

In [ ]:
sns.distplot(pdf.Owned, bins=owned_fjb, norm_hist=True, kde=False)

Now let's create a map using the Fisher Jenks classification (remember the map may take a while to plot because GeoPandas is using the `fisher_jenks` function from PySAL):

In [ ]:
gdf.plot(column='Owned', scheme='fisher_jenks', k=5)

3. Compare the map you just made to the one we made using quintiles. Do you think their representation of the spatial distribution is very different?

> Your answer here.

Let's see if a map created using an `equal_interval` classification looks any different. Combine the information above to write all of the code in one cell to output the classification, histogram and map:

In [ ]:
???

Allowing for differences in the colour mapping, your map should look like this:

![Equal Interval Map](https://github.com/kingsgeocomp/geocomputation/raw/master/img/Equal_Interval_Map.png)

This visualisation of the spatial distribution of _Owned_ households using the Equal Interval classification looks quite a bit different to the quantile classification. This is mainly because there are fewer LSOAs classified in the highest (and lowest) classes, meaning that the extreme values are more obvious on the map. You should also be able to see the difference between the shape of the histogram for a Equal Interval classification versus the other classifications.  

#### Task 4

For the next two tasks, you have already produced all the information you need (so no more code needed). Hopefully, answering these tasks will reinforce your understanding about how the three classifications we have tried differ.

1. How many LSOAs are in the top class for each of the three classifications:

> Your answer here:
    

2. What is the interval for the top class for each of the three classifications:

> Your answer here:
    

3. Create three more maps for the _Owned_ variable using the three classifications (Quantiles, Fisher-Jenks, and Equal Interval), but this time use **seven** classes in each.  

In [ ]:
???

In [ ]:
???

In [ ]:
???

4. Now create three maps of _GreenspaceArea_ using each of the three classification methods. For each of these three maps, also output histograms. 

In [ ]:
qu = np.insert(mp.Quantiles(gdf.GreenspaceArea.values, k=7).bins, 0, gdf.GreenspaceArea.min())
???

In [ ]:
???

In [ ]:
???

5. Why are the differences between the three maps for _GreenspaceArea_ more obvious than for the _Owned_ variable we looked at above? 

> Your answer here.

## Map Modifications

GeoPandas uses much of its plotting functionality from matplotlib, and we can use this to modify and hopefully improve how well our maps communicate. Seaborn also uses matplotlib so hopefully some of the code patterns below will look familiar. 

First, in the classified choropleth maps above when we included a legend we found that it overlapped with the actual map. Legends are still pretty rudimentary in GeoPandas and moving them is not straight-forward (if you can find an easy way, please share!). So currently the easiest way to ensure the legend does not overlap the map itself is to make the size of our plot larger using the `figsize` option when creating a subplot:

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(12, 10))   # create figure and axes for Matplotlib 
gdf.plot(column='Owned', scheme='quantiles', k=5, legend=True, ax=ax1)  #include ax argument!

Alternatively, we could keep the plot smaller but change the axis limits of the plot so that there is more white space on one side of the map in which the legend can be drawn:

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(8, 6))   # create figure and axes for Matplotlib
gdf.plot(column='Owned', scheme='quantiles', k=5, legend=True, ax=ax1) #include ax argument!
ax1.set_xlim(500000,586000)   # you can play with these values and see what happens!
plt.show()

In the code above, I just used trial-and-error until I found the right values for `set_xlim` given the size of the plot I wanted. These workarounds aren't the most elegant or efficient but they work for making single maps. 

Using more functionality from matplotlib we can make our map look much cleaner: 

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(15, 12))   
gdf.plot(column='Owned', scheme='quantiles', k=5, legend=True, ax=ax1, edgecolor='grey', linewidth=0.2)  #change line style
ax1.axis('off') #don't plot the axes (bounding box)
ax1.set_title('Ownership', fontdict={'fontsize':'20', 'fontweight':'3'})  #provide a title
ax1.annotate('Source: London Datastore (2011)',
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')  #add source info on the image itself
ax1.get_legend().set_title("Households")  #set the legend title

In the code above check you can see where we:
1. change the colour and width of LSOA outlines so they are easier to see
2. set the figure title to communicate the variable being mapped
3. set the legend title to communicate the units of the data being mapped
4. add an annotation show the source of the data for the map.

Here's another trick to modify the legend labels:

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(15, 12))   
ax1 = gdf.plot(column='Owned', scheme='quantiles', k=5, legend=True, ax=ax1, edgecolor='grey', linewidth=0.2)  #change line style
ax1.axis('off')  #don't plot the axes (bounding box)
ax1.set_title('Ownership', fontdict={'fontsize': '20', 'fontweight' : '3'})  #provide a title
ax1.annotate('Source: London Datastore (2011)',xy=(0.1, 0.1),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')  #add source info on the image itself

#edit legend labels
leg = ax1.get_legend()
leg.get_texts()[0].set_text('5 - 199')
leg.get_texts()[1].set_text('199 - 291')
leg.get_texts()[2].set_text('291 - 366')
leg.get_texts()[3].set_text('366 - 452')
leg.get_texts()[4].set_text('452 - 710')
leg.set_title("Households")

#### Task 5

Have a think about how you could change the hard-coded legend labels into a loop that dynamically formats the legend as integers/floats with no decimal places:

```python
leg = ax1.get_legend()
leg.get_texts()[0].set_text('5 - 199')
leg.get_texts()[1].set_text('199 - 291')
leg.get_texts()[2].set_text('291 - 366')
leg.get_texts()[3].set_text('366 - 452')
leg.get_texts()[4].set_text('452 - 710')
```
As a hint, part of your code should look like this:
```python
    bounds = [float(x) for x in i.get_text().split(' - ')]
    b_txt  = "{0:.0f} - {1:.0f}".format(bounds[0], bounds[1])
```
If you don't understand what this is doing, I'd suggest adding comments and printing out variables as you go...

In [ ]:
???

Above we saw how we can change the axis limits to zoom out to make room for the legend. But we can also use this to zoom in on our map to a specific location we want to show in more detail:

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(8, 6))   
gdf.plot(column='Owned', scheme='quantiles', k=5, legend=True, ax=ax1, edgecolor='grey', linewidth=0.2) 
ax1.set_xlim(535000,550000)   #play with these values
ax1.set_ylim(177000,190000)   #play with these values

### Saving maps!

There are a few  ways you can get your maps out of a notebook and into a Word document (_e.g._ to use in your reports). One might be to right-click (or Cmd-click on Mac), select 'copy image' and then paste directly into Word; however, this doesn't always work very well, so a better way is to save your map directly to a file and then insert it in Word. 

You could save your map to a file by (again) right-clicking (or Cmd-clicking on Mac) and select 'Save Image As'. Then you could insert the saved file into your Word document (by going to Insert -> Pictures). 

However, probably the best way to save a map to file is to use code, and note how this is the _same_ code you'd use for a Seaborn figure:

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(8, 6))   
gdf.plot(column='Owned', scheme='quantiles', k=5, legend=True, ax=ax1, edgecolor='grey', linewidth=0.2) 
ax1.set_xlim(535000,550000)   
ax1.set_ylim(177000,190000)   

# And here is the key line of code
# Note that you can also save to a PDF 
# though there's no guarantee that the
# output will be vector...
fig.savefig('map_export.png', dpi=300)

Go and check your working directory (usually where you have saved this notebook) to check the image was created!

You can [read more about the `savefig` method](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.savefig.html) to see more options available, but `dpi` is the argument that will change the size of your output image. Remember, files will be saved to whatever your current working directory is, but you can include a path if you want to save elsewhere. 

### Map Layers 

So far we have looked at only a single variable on a map, but it is possible to present data for multiple variables by layering them on top of one another. For Geopackage files Geopandas can read these directly from a URL (**very cool**) with no need to download and unzip the data first. Since a Shapefile is actually made up of at _least_ four separate files, this one needs to _first_ be saved locally and unzipped in before it can be loaded. So for obvious reasons we rather like the new-ish Geopackage format.

Here's code to do the download the data and write it to a directory (creating it if it does not exist):

In [ ]:
import os
os.makedirs('data', exist_ok=True)

# Load Water GeoPackage
w_path = os.path.join('data','Water.gpkg')
if not os.path.exists(w_path):
    water = gpd.read_file('https://github.com/kingsgeocomp/geocomputation/raw/master/data/src/Water.gpkg')
    water.to_file(w_path, driver='GPKG')
    print("Downloaded Water.gpkg file.")
else:
    water = gpd.read_file(w_path)
    print("Loaded Water.gpkg file.")

# Boroughs GeoPackage
b_path = os.path.join('data','Boroughs.gpkg')
if not os.path.exists(b_path):
    boroughs = gpd.read_file('https://github.com/kingsgeocomp/geocomputation/raw/master/data/src/Boroughs.gpkg')
    boroughs.to_file(b_path, driver='GPKG')
    print("Downloaded Boroughs.gpkg file.")
else:
    boroughs = gpd.read_file(b_path)
    print("Loaded Boroughs.gpkg file.")
    
# Greenspace GeoPackage
g_path = os.path.join('data','Greenspace.gpkg')
if not os.path.exists(g_path):
    greenspace = gpd.read_file('https://github.com/kingsgeocomp/geocomputation/raw/master/data/src/Greenspace.gpkg')
    greenspace.to_file(g_path, driver='GPKG')
    print("Downloaded Greenspace.gpkg file.")
else:
    greenspace = gpd.read_file(g_path)
    print("Loaded Greenspace.gpkg file.")

print("Done.")

See how this function allows us to download data to a particular directory:

In [ ]:
greenspace.head(3)

If the data read successfully, you should be able to see the top three lines of the data, with a _geometry_ column on the far right. 

Let's see if we can plot this:

In [ ]:
greenspace.plot(color='green')

Hopefully you got something that looked a bit like a map (well, maybe just a bunch of green blobs...). If so, then we have loaded our data properly as a GeoPandas DataFrame. 

So, let's try to plot our greenspace (parks) data on top of the Owned data (we'll also zoom in so we can see the result in a little more detail):

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(12, 10))  #setup the figure

base = gdf.plot(ax=ax1, column='Owned', scheme='quantiles', k=5, cmap='OrRd')         #plot the Owned data layer on ax1
parks = greenspace.plot(ax=ax1, color=(0.1, 0.5, 0.2), alpha=0.75, edgecolor='green') #plot the parks data later on ax1

ax1.set_xlim(535000,550000)   #zoom
ax1.set_ylim(177000,190000)   #zoom
plt.show()

The key here is that both plots use `ax=ax1` ensuring that they are plotted on the same set of axes. The first plot is on the bottom, then additional plots are layered on top. 

Try plotting but removing `ax=ax1` from one of the plot calls to see what happens when we don't plot on the same axes: 

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(12, 10))  #setup the figure

base = gdf.plot(ax=ax1, column='Owned', scheme='quantiles', k=5, cmap='OrRd')       #plot the Owned data layer on ax1
parks = greenspace.plot(color=(0.1, 0.5, 0.2), alpha=0.75, edgecolor='green')         #plot the parks data layer but NOT on ax1

ax1.set_xlim(535000,550000)   #zoom
ax1.set_ylim(177000,190000)   #zoom
plt.show()

Finally, we can create a map template that we could use across a number of outputs in order to ensure that each one is fairly consistent:

In [ ]:
def plt_ldn(w=water, b=boroughs):
    fig, ax = plt.subplots(1, figsize=(14, 12))
    w.plot(ax=ax, color='#79aef5', zorder=2)
    b.plot(ax=ax, edgecolor='#cc2d2d', facecolor='None', zorder=3)
    ax.set_xlim([502000,563000])
    ax.set_ylim([155000,201500])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    return fig, ax

fig, ax = plt_ldn()
gdf.plot(column='Owned', scheme='quantiles', k=5, cmap='YlGnBu', legend=True, alpha=0.9, zorder=1, ax=ax) 

#### Task 6

Duplicate and modify the `plt_ldn` function to include greenspace as well, and then see if you can write a _loop_ that iterates over White, Asian, and Black, creating and saving a Fisher Jenks map (with title) of each. You should _also_ implement the tricks we've seen above in terms of adding a title, formatting the legend appropriately (you might even want to implement this as a function), etc.

In [ ]:
???

## Merging Spatial and Non-Spatial Data

Finally, let's consider the situation where we don't have an existing (single) dataset that contains geometry, but rather we have two files:
1. a file containing geometry for spatial elements
2. a file containing data for spatial elements but no geometry

In this case, as long as we have a common identifier for each row in the two files, we can use the Pandas `merge` function to join them together. 

For example, let's assume the data we have are:
1. A Geopackage delineating LSOAS in and around London 
2. A csv file containing air quality data for each of our London LSOAs

First, let's download the shapefile data (using the `download_gsa_geodata` function we defined above):

In [ ]:
lsoas = gpd.read_file('https://github.com/kingsgeocomp/geocomputation/raw/master/data/src/LSOAs.gpkg', low_memory=False)
print(lsoas.columns.values)
lsoas.plot()

Looks okay, but note the map - it looks like there might be more LSOAs than in our previous maps? Let's check the dimensions of this DataFrame:

In [ ]:
print(lsoas.shape)

Hopefully you can see that there are more rows in this DataFrame than the one we have been using before. 

Next, let's load the csv data file as a Pandas DataFrame and look at the top few lines:  

In [ ]:
aq = pd.read_csv(
    'https://github.com/kingsgeocomp/geocomputation/blob/master/data/LSOA_AirQuality.csv.gz?raw=true',
    compression='gzip', low_memory=False) 

aq.head(3)

And check the dimensions of this DataFrame:

In [ ]:
print(aq.shape)

There's certainly more rows in one DataFrame than the other. 

But look at the column names of our two data files; can you see a common column between them?

The _LSOA11CD_ and _lsoa11cd_ columns contain the same codes so we can use these to merge the data into a single DataFrame. We'll do an _inner join_ (see week 7) with the Air Quality data on the left; this is because there are more rows in the shapefile than the Air Quality data, so we only want to retain rows from the shapefile where we have air quality data: 

In [ ]:
lsoa_aq = pd.merge(aq, lsoas, how="inner", left_on='LSOA11CD', right_on='lsoa11cd')

Let's check what the top of our new DataFrame looks like:

In [ ]:
lsoa_aq.head()

And let's check the dimensions of our new DataFrame:

In [ ]:
print(lsoa_aq.shape)

So we have the same number of rows as the Air Quality dataframe but the number of columns is the total of the two original DataFrames (check you can from the outputs above how we know this). 

And finally, let's check what _type_ of DataFrame we've created:

In [ ]:
print(type(lsoa_aq))

We've created a Pandas DataFrame. So we need to convert to a GeoPandas DataFrame if we want to make some maps; to do this we need to specify what column contains the geometry:

In [ ]:
lsoa_aq = lsoa_aq.set_geometry('geometry')
print(type(lsoa_aq))

And now let's try to plot it spatially:

In [ ]:
lsoa_aq.plot('NO2max')

Hopefully that was successful! Does the this map have the same shape as our previous maps?

And now we have created this new GeoPandas DataFrame we can start to examine the Air Quality data spatially. For example: 

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(15, 12))   
ax1 = lsoa_aq.plot(column='NOxmean', 
                   scheme='quantiles', 
                   k=7, 
                   legend=True, 
                   ax=ax1, 
                   edgecolor='grey',    
                   linewidth=0.2)       

#### Task 7

1. _Join_ the `lsoa_aq` and `gdf` files so that you can access both demographic and pollution data. I've given you code to get started but you will also need to to do some careful reading of the error messages. 

In [ ]:
# You only want to run this _once_ since it manually 
# sets the index so that we can use the _join_ syntax
lsoa_aq.set_index('LSOA11CD', inplace=True)
gdf.set_index('LSOA11CD', inplace=True)

In [ ]:
???

2. Now create a filtered data frame (let's call it `fdf`) containing only LSOAs with more than 2,000 usual residents, and check that it worked by creating a map.

In [ ]:
???

3. Now map mean PM2.5 using five equal intervals for the filtered data frame. Use a grey background for the rest of London (you might want to check out: [this file](https://github.com/kingsgeocomp/geocomputation/raw/master/data/src/London.gpkg)) and add the roads ([from this file](https://github.com/kingsgeocomp/geocomputation/raw/master/data/src/Roads.gpkg)) in red. You'll need to do some layering for this so I'd suggest copying _some_ of the code out of the `plt_ldn` and modifying it accordingly, but it's probably not worth a whole new function. You will also need to work out how to specify a light grey, since the default is quite dark... suggest googling matplotlib colors.

The final map should look something like the one below. 

In [ ]:
???

![ownership](https://github.com/kingsgeocomp/geocomputation/raw/master/img/PM25_in_Populated_LSOAs.png)

4. Create a map of _Mean PM10_ for central London with roads layered on top. Classify the air quality data into five quantiles and zoom in towards central London. Your final map should look something like the map below. _Note_ that you will need to investigate how to do both mathematical symbols and superscripts in matplotlib/python.

In [ ]:
???

![MeanPM10 in Central London](https://github.com/kingsgeocomp/geocomputation/raw/master/img/PM10central.png)

5. How well does the (incomplete) road map align with air quality would you say?

> Your answer here.

#### More Fun!

You can find some more nice examples and applications from [GeoHackWeek](https://geohackweek.github.io/vector/04-geopandas-intro/)!

### Getting More Help/Applications

A great resource for more help and more examples is Dani Arribas-Bel's _Geographic Data Science_ module: he has put all of his [module practicals online](http://darribas.org/gds17/) (as we have too), and you might find that something that he does makes more sense to you than what we've done... check it out!

## Credits!

#### Contributors:
The following individuals have contributed to these teaching materials: Jon Reades (jonathan.reades@kcl.ac.uk), James Millington (james.millington@kcl.ac.uk)

#### License
These teaching materials are licensed under a mix of [The MIT License](https://opensource.org/licenses/mit-license.php) and the [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 license](https://creativecommons.org/licenses/by-nc-sa/4.0/).

#### Acknowledgements:
Supported by the [Royal Geographical Society](https://www.rgs.org/HomePage.htm) (with the Institute of British Geographers) with a Ray Y Gildea Jr Award.

#### Potential Dependencies:
This notebook may depend on the following libraries: pandas, matplotlib, seaborn